# Imports

In [1]:
import sqlite3
import pandas as pd
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt

SQL_LITE_DB_PATH = "D:\\Repos\\ut-health\\mimic-iii-gz\\mimic3.db"
CREATE_PYVIS_GRAPHS = False

# Connect to the SQLite database in the file at the given path d:\data\mimic.db
conn = sqlite3.connect(SQL_LITE_DB_PATH)

# print version of sqlite
cursor = conn.cursor()
cursor.execute("SELECT sqlite_version()")
print("SQLite version: %s" % cursor.fetchone())

# close connection
conn.close()

# print pandas version
print("Pandas version: %s" % pd.__version__)

# print matplotlib version
print(f"Matplotlib version: {matplotlib.__version__}")

print(f"NetworkX version: {nx.__version__}")

# Set pandas display options to show all columns and rows without truncation
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
print("\nRemoved truncation of columns")


SQLite version: 3.49.1
Pandas version: 2.2.3
Matplotlib version: 3.10.1
NetworkX version: 3.4.2

Removed truncation of columns


# Load Data

In [2]:
# load patient data
conn = sqlite3.connect(SQL_LITE_DB_PATH)

query = "SELECT * FROM patients"
patients_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM d_icd_diagnoses"
d_icd_diagnoses_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM diagnoses_icd"
diagnoses_icd_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM admissions"
admissions_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM procedures_icd"
procedures_icd_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM d_icd_procedures"
d_icd_procedures_df = pd.read_sql_query(query, conn)

conn.close()

# Subselect size of data
Reduces the number of patients, picks the latest admission (hadm_id) for the patient and picks only those diagnoses belonging to the hospital admission.

In [ ]:
# print the number of patients and diagnoses
print(f"Number of patients: {len(patients_df)}")
print(f"Number of diagnoses: {len(diagnoses_icd_df)}")
print(f"Number of admissions: {len(admissions_df)}")
print(f"Number of procedures: {len(procedures_icd_df)}")

# patient fraction of the full 46k patient dataframe to select for processing
pt_frac = 0.0001  # gives 5 patients, 52 diagnoses
pt_frac = 0.0002  # gives 9 patients, 110 diagnoses
pt_frac = 0.0003  # gives 14 patients, 188 diagnoses
pt_frac = 0.0015  # gives 70 patients, 813 diagnoses
pt_frac = 0.01    # gives 465 patients, 5085 diagnoses
pt_frac = 0.02    # gives 930 patients, 9939 diagnoses
pt_frac = 0.05    # gives 2326 patients, 24959 diagnoses

pt_frac = 0.0002

# select 0.01% of the patients randomly
patients_df = patients_df.sample(frac=pt_frac, random_state=1)

# add DOB to admissions_df
admissions_df = admissions_df.merge(patients_df[['SUBJECT_ID', 'DOB']], left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='inner', suffixes=('_adm', '_pt'))

# sort admissions_df by SUBJECT_ID and ADMITTIME in ascending order
admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'])
admissions_df = admissions_df.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])

# keep the newest admission (HADM_ID) for each patient, drop the rest of the admissions
admissions_df = admissions_df.groupby('SUBJECT_ID').last().reset_index()

# keep only diagnoses that belong to HADM_IDs in admissions_df
diagnoses_icd_df = diagnoses_icd_df[diagnoses_icd_df['HADM_ID'].isin(admissions_df['HADM_ID'])]

# keep only procedures that belong to HADM_IDs in admissions_df
procedures_icd_df = procedures_icd_df[procedures_icd_df['HADM_ID'].isin(admissions_df['HADM_ID'])]

# print the number of patients and diagnoses
print(f"Updated number of patients: {len(patients_df)}")
print(f"Updated number of diagnoses: {len(diagnoses_icd_df)}")
print(f"Updated number of admissions: {len(admissions_df)}")
print(f"Updated number of procedures: {len(procedures_icd_df)}")

Number of patients: 46520
Number of diagnoses: 651047
Number of admissions: 58976
Number of procedures: 240095
Updated number of patients: 930
Updated number of diagnoses: 9939
Updated number of admissions: 930
Updated number of procedures: 3704


# Feature Engineering
Adds age bucket feature and hadm_id to patient dataframe

In [4]:
# log data
print(f"Number of patients: {len(patients_df)}")

# convert to integer from datetime type to calculate age (without overflow)
admissions_df['DOB'] = admissions_df['DOB'].astype('datetime64[s]').astype(int)
admissions_df['ADMITTIME'] = admissions_df['ADMITTIME'].astype('datetime64[s]').astype(int)

# calculate age at admission
admissions_df['AGE'] = (admissions_df['ADMITTIME'] - admissions_df['DOB']) / 86400 / 365

# create age intervals in 5 year increments, of type int
admissions_df['AGE_BUCKET'] = (admissions_df['AGE'] // 5).astype(int) * 5

# set age bucket to 95 where age is greater than 90
admissions_df.loc[admissions_df['AGE_BUCKET'] > 90, 'AGE_BUCKET'] = 95

# add AGE_BUCKET to patients_df
patients_df = patients_df.merge(admissions_df[['SUBJECT_ID', 'HADM_ID', 'AGE_BUCKET']], left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='inner', suffixes=('_pt', '_adm'))

print(f"Updated number of patients (with age bucket): {len(patients_df)}")

Number of patients: 930
Updated number of patients (with age bucket): 930


In [5]:
procedures_icd_df.head(5)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
1156,1934,11930,124220,1,415
1157,1935,11930,124220,2,5411
1158,1936,11930,124220,3,387
1159,1937,11930,124220,4,9672
1160,1938,11930,124220,5,3404


In [6]:
patients_df.head(5)

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,AGE_BUCKET
0,3850,4074,M,2121-08-01 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,1,137421,80
1,44143,90889,M,2095-04-29 00:00:00,None,None,None,0,168745,55
2,39453,72753,M,2042-06-01 00:00:00,None,None,None,0,144029,85
3,37377,64908,M,2108-04-22 00:00:00,None,None,None,0,172851,75
4,38824,70273,F,2090-05-28 00:00:00,None,None,None,0,102912,75


# Create Network Graph

## Create nodes: patients

In [7]:
# Create a graph
G = nx.MultiGraph()
print(f"Number of nodes in the graph: {G.number_of_nodes()}")
i = 0

for _, row in patients_df.iterrows():

    # Add a node with the patient ID as the node identifier and gender as an attribute, also add dob as an attribute
    G.add_node(row['SUBJECT_ID'], gender=row['GENDER'], age_bucket=row['AGE_BUCKET'], hadm_id=row['HADM_ID'])

    i+=1
    if i % 2000 == 0:
        print(f"Added {i} patient nodes, percent: {i / len(patients_df) * 100:.2f}%")

# number of nodes in the graph
print(f"Number of nodes in the graph: {G.number_of_nodes()}")

Number of nodes in the graph: 0
Number of nodes in the graph: 930


## Create procedures to patient mappings

In [8]:
# count unique ICD9 codes in the procedures dataframe
unique_icd9_codes = procedures_icd_df['ICD9_CODE'].nunique()
print(f"Number of unique ICD9 codes: {unique_icd9_codes}")

# print shape of procedures dataframe
print(f"Shape of procedures dataframe: {procedures_icd_df.shape}")


Number of unique ICD9 codes: 501
Shape of procedures dataframe: (3704, 5)


In [9]:
procedures_icd_df.head(5)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
1156,1934,11930,124220,1,415
1157,1935,11930,124220,2,5411
1158,1936,11930,124220,3,387
1159,1937,11930,124220,4,9672
1160,1938,11930,124220,5,3404


## Create edges: procedures connecting patients

In [10]:
# print the number of edges in the graph
print(f"Number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

procs_to_patients = procedures_icd_df.groupby('ICD9_CODE')['SUBJECT_ID'].apply(set).to_dict()

# add diagnosis/icd9 edge between patients
for icd9_code, patients in procs_to_patients.items():

    # if this code has only one patient, there is no edge, skip it
    if len(patients) < 2:
        continue

    # get short_title from d_icd_procedures_df
    short_title = d_icd_procedures_df[d_icd_procedures_df['ICD9_CODE'] == icd9_code]['SHORT_TITLE'].values

    if len(short_title) == 0:
        short_title = "Unknown"
    else:
        # convert to a string
        short_title = short_title[0]

    # if there are more than 1 patients with the same diagnosis, add edges between them
    patients = list(patients)

    combos = [(patients[i], patients[j]) for i in range(len(patients)) for j in range(i + 1, len(patients))]
    # print(f"Number of edges to add: {len(combos)} for icd {icd9_code} ({short_title})")
    G.add_edges_from(combos, icd9=icd9_code, short_title=short_title, color='orange', type='procedures')

# print the number of edges in the graph
print(f"Number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

Number of edges: 0
Number of nodes: 930
Number of edges: 101097
Number of nodes: 930


## Create diagnosis to patient mappings

In [11]:
# group diagnoses by icd9 codes, and create a dictionary with each icd9 code as the key and the subject_ids as the values
diagnoses_to_patients = diagnoses_icd_df.groupby('ICD9_CODE')['SUBJECT_ID'].apply(set).to_dict()

# print 1 random key and value from the dictionary
print(f"Sample keys and values from the dictionary:")
print(f"\t\t{list(diagnoses_to_patients.items())[12]}")
print(f"\t\t{list(diagnoses_to_patients.items())[15]}")

# print size of the dictionary and counts of values
print(f"Count of the number of keys (icd9s) in the dictionary: {len(diagnoses_to_patients)}")
print(f"Count of the number of values (subject_ids) in all keys: {sum(len(v) for v in diagnoses_to_patients.values())}")

Sample keys and values from the dictionary:
		('03819', {59877, 9768, 3273, 22772, 1143})
		('03840', {84011})
Count of the number of keys (icd9s) in the dictionary: 1758
Count of the number of values (subject_ids) in all keys: 9937


### Validation code

In [12]:
# Validation
unique_subject_ids = len(set([item for sublist in diagnoses_to_patients.values() for item in sublist]))

# count number of unique subject_ids across all values in the dictionary
print(f"Count of the number of unique subject_ids across all keys: {unique_subject_ids}")

# print number of patients with diagnosis 0389
print(f"Number of patients with diagnosis 0389: {len(diagnoses_to_patients.get('0389', []))}")

# assert that the number of unique subject_ids is equal to the number of patients in the patients_df dataframe
assert unique_subject_ids == len(patients_df), f"Number of unique subject_ids in the dictionary ({unique_subject_ids}) is not equal to the number of patients in the patients_df dataframe ({len(patients_df)})"


Count of the number of unique subject_ids across all keys: 930
Number of patients with diagnosis 0389: 54


## Create edges:  diagnoses connecting patients

In [13]:
edge_count = len(G.edges())

# print the number of edges in the graph
print(f"Number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

i=0
e_added = 0
ONCE = True

# add diagnosis/icd9 edge between patients
for icd9_code, patients in diagnoses_to_patients.items():

    # get SHORT_TITLE from d_icd_diagnoses_df for the icd9 code
    short_title = d_icd_diagnoses_df[d_icd_diagnoses_df['ICD9_CODE'] == icd9_code]['SHORT_TITLE'].values

    if len(short_title) == 0:
        short_title = "Unknown"
    else:
        # convert to a string
        short_title = short_title[0]

    if ONCE:
        print(f"ICD9 code: {icd9_code}, short title: {short_title}")
        ONCE = False

    if len(patients) < 2:
        continue

    # if there are more than 1 patients with the same diagnosis, add edges between them
    patients = list(patients)

    combos = [(patients[i], patients[j]) for i in range(len(patients)) for j in range(i + 1, len(patients))]
    # print(f"Number of edges to add: {len(combos)} for icd {icd9_code} ({short_title})")
    G.add_edges_from(combos, icd9=icd9_code, short_title=short_title, color='blue', type='diagnosis')

    i += 1
    if i % 4000 == 0:
        print(f"Added {i} edges, percent: {i / len(diagnoses_to_patients) * 100:.2f}%")


# print the number of edges in the graph
print(f"Number of edges added: {len(G.edges()) - edge_count}")
print(f"Total number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

Number of edges: 101097
Number of nodes: 930
ICD9 code: 0030, short title: Salmonella enteritis
Number of edges added: 239268
Total number of edges: 340365
Number of nodes: 930


# Create Similarity Graphs

## Create patient-icd9 proc encoded dataframe

In [14]:
procedures_icd_df.head(5)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
1156,1934,11930,124220,1,415
1157,1935,11930,124220,2,5411
1158,1936,11930,124220,3,387
1159,1937,11930,124220,4,9672
1160,1938,11930,124220,5,3404


In [15]:
print(f"Number of patients: {len(patients_df)}")

patient_to_icd9proc = procedures_icd_df.groupby('SUBJECT_ID')['ICD9_CODE'].apply(set).to_dict()

# get unique list of icd9 codes for all patients
all_icd9_codes = set()
for codes in patient_to_icd9proc.values():
    all_icd9_codes.update(codes)

# create dataframe with icd9 codes as columns and patients as rows
icd9_proc_df = pd.DataFrame(columns=list(all_icd9_codes), index=patients_df['SUBJECT_ID'])

# fill the dataframe with the icd9 codes
for patient_id, codes in patient_to_icd9proc.items():
    icd9_proc_df.loc[patient_id] = [1 if code in codes else 0 for code in icd9_proc_df.columns]

# fill the rest of the dataframe with 0s
icd9_proc_df.fillna(value=0, inplace=True)

icd9_proc_df.head(5)


Number of patients: 930


C:\Users\Razal\AppData\Local\Temp\ipykernel_10748\1510776575.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  icd9_proc_df.fillna(value=0, inplace=True)


,5121,5122,4103,4105,13,14,9229,5136,17,22,8961,34,4131,4132,40,41,42,44,45,46,47,48,7817,8244,8245,8246,2102,2100,59,61,63,5184,5185,66,5187,5188,5183,3142,2121,5198,91,93,94,102,3179,110,114,8309,118,8314,2172,9341,124,4223,3199,8321,3201,131,126,127,4233,9354,5259,139,9357,8332,9359,4242,8339,151,8344,8345,3228,159,4258,8365,9390,4269,3249,9396,184,4514,8377,4515,8382,5310,4289,8385,4292,202,8395,204,206,8401,9427,212,4311,8411,8412,8415,4319,8417,5349,5351,234,239,5361,242,243,4341,9462,3321,3322,3323,3324,3327,8451,8452,2309,8456,270,2319,3343,3348,3349,8472,282,5411,294,5425,309,3725,311,4413,4414,9543,331,3404,3403,9547,4432,3409,5459,9633,5462,4439,5463,4443,4444,3424,353,5472,3425,359,4461,370,5491,5495,3451,3452,5502,5503,2432,387,9604,9605,390,9607,391,9608,9606,403,5523,6549,4502,3479,8604,8605,3484,415,8607,6561,4513,9634,3491,4516,9638,3492,8611,4519,9635,4523,4524,4525,8622,5551,430,3499,8626,8628,3512,9656,9659,4542,4543,3521,8841,3523,3522,3524,9671,9672,8842,3533,8844,4562,8659,3539,8662,472,5593,4572,4573,8669,4576,8674,4579,4582,3561,8849,8683,8682,9705,4591,4593,3571,500,4595,503,9723,4603,7679,8703,4610,4611,9732,4613,9737,7691,3596,7692,4621,4620,9744,3601,8721,5651,3605,3606,3607,4632,3611,3612,3613,3614,3615,3616,4639,544,540,539,8741,543,4652,8751,560,8753,8754,6712,9788,4671,4673,4675,7749,4679,8775,9802,7763,7764,4693,598,9815,7768,7769,7770,4694,8801,7779,5732,4709,7784,7789,4719,640,3712,7815,3721,3722,3723,5771,8845,3726,8847,8848,3727,7819,3731,8852,8853,8854,8855,8856,8857,3734,3733,3736,8864,8866,5795,5794,8872,8874,7851,9900,7855,9904,9905,9906,9907,3761,9910,9911,6839,7863,9914,9915,3772,7865,7867,9920,3777,3778,9921,9924,9925,6849,3783,9928,9929,3787,3789,722,3794,9939,9941,4823,4824,8919,7901,3808,9952,7906,9955,4835,7909,3814,4836,9960,9961,3818,9962,8938,741,3822,8945,9969,9971,9972,6901,6902,9977,3834,7931,7932,7933,9982,8959,7935,9983,8960,8962,8964,3845,3844,7936,3848,8968,7939,782,3861,3863,7962,7965,7966,7972,7975,6952,3880,3885,3888,3889,3891,3892,3893,3895,3897,3899,8006,843,8013,8016,3922,3923,851,3927,3929,3930,3931,3932,863,3941,3949,3950,3951,881,8051,3956,3957,886,887,3961,3963,3964,3965,3971,3972,3973,3974,3976,3979,8076,5011,5012,5013,3990,8086,3995,3998,8099,5029,8102,8103,8101,8105,8106,4011,8108,8104,8113,4019,4021,9755,5059,5061,966,8135,8138,8152,8154,8162,8163,8164,5101,7934,5103,5104,5102,5110,5111,5114,8191
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
90889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Create patient-icd9 diagnosis encoded dataframe
This graph contains patient nodes, and only those patients are connected with edges that are similar to each other

In [16]:
print(f"Number of patients: {len(patients_df)}")

patient_to_icd9 = diagnoses_icd_df.groupby('SUBJECT_ID')['ICD9_CODE'].apply(set).to_dict()

# get unique list of icd9 codes for all patients
all_icd9_codes = set()
for codes in patient_to_icd9.values():
    all_icd9_codes.update(codes)

# create dataframe with icd9 codes as columns and patients as rows
icd9_df = pd.DataFrame(columns=list(all_icd9_codes), index=patients_df['SUBJECT_ID'])

# fill the dataframe with 1s and 0s
for patient_id, codes in patient_to_icd9.items():
    icd9_df.loc[patient_id] = [1 if code in codes else 0 for code in icd9_df.columns]

icd9_df.head(5)

Number of patients: 930


2858 4474 29630 8221 44481 71595 6961 4260 80507 56409 81501 36281  \
SUBJECT_ID                                                                      
4074          0    0     0    0     0     0    0    0     0     0     0     0   
90889         0    0     0    0     0     0    0    0     0     0     0     0   
72753         0    0     0    0     0     0    0    0     0     0     0     0   
64908         0    0     0    0     0     0    0    0     0     0     0     0   
70273         0    0     0    0     0     0    0    0     0     0     0     0   

           77083 86803 78951 E9330 E8782 4210 80842 7856 8670 78659 7249  \
SUBJECT_ID                                                                 
4074           0     0     0     0     0    0     0    0    0     0    0   
90889          0     0     0     0     1    0     0    0    0     0    0   
72753          0     0     0     0     0    0     0    0    0     0    0   
64908          0     0     0     0     0    0     0    0    0     0    0   
70273          0     0     0     0     0    0     0    0    0     0    0   

           27669 5110 0971 5070 6000 86403 70709 5302 V1041 5722 6805 30391  \
SUBJECT_ID                                                                    
4074           0    0    0    0    0     0     0    0     0    0    0     0   
90889          0    0    0    0    0     0     0    0     0    0    0     0   
72753          0    0    0    0    0     0     0    0     0    0    0     0   
64908          0    0    0    0    0     0     0    0     0    0    0     0   
70273          0    0    0    0    0     0     0    0     0    0    0     0   

           78821 76519 9534 7231 9708 5718 30591 36221 E8796 220 5759 V5867  \
SUBJECT_ID                                                                    
4074           0     0    0    0    0    0     0     0     0   0    0     0   
90889          0     0    0    0    0    0     0     0     0   0    0     0   
72753          0     0    0    0    0    0     0     0     0   0    0     0   
64908          0     0    0    0    0    0     0     0     0   0    0     0   
70273          0     0    0    0    0    0     0     0     0   0    0     0   

           56983 7242 2440 2409 5730 23871 03810 0090 70703 0311 5711 5401  \
SUBJECT_ID                                                                   
4074           0    0    0    0    0     0     0    0     0    0    0    0   
90889          0    0    0    0    0     0     0    0     0    0    0    0   
72753          0    0    0    0    0     0     0    0     0    0    0    0   
64908          0    0    0    0    0     0     0    0     0    0    0    0   
70273          0    0    0    0    0     0     0    0     0    0    0    0   

           53560 V3001 41001 45620 86504 78065 75470 1830 7245 8083 3051  \
SUBJECT_ID                                                                 
4074           0     0     0     0     0     0     0    0    0    0    0   
90889          0     0     0     0     0     0     0    0    0    0    0   
72753          0     0     0     0     0     0     0    0    0    0    0   
64908          0     0     0     0     0     0     0    0    0    0    0   
70273          0     0     0     0     0     0     0    0    0    0    0   

           78451 E9342 1561 30540 82521 43330 79319 1976 V430 496 41092 7824  \
SUBJECT_ID                                                                     
4074           0     0    0     0     0     0     0    0    0   0     0    0   
90889          0     0    0     0     0     0     0    0    0   0     0    0   
72753          0     0    0     0     0     0     0    0    0   0     0    0   
64908          0     0    0     0     0     0     0    0    0   1     0    0   
70273          0     0    0     0     0     0     0    0    0   1     0    0   

           5370 2639 99933 2532 41011 45340 1982 7756 67202 42789 78551 78079  \
SUBJECT_ID                                                                      

In [17]:
# merge the two dataframes on the index (SUBJECT_ID)
icd9_df = icd9_df.merge(icd9_proc_df, left_index=True, right_index=True, suffixes=('_diag', '_procs'))

# fill the rest of the dataframe with 0s
icd9_df.fillna(0, inplace=True)

icd9_df.head(5)


2858 4474 29630 8221 44481 71595 6961 4260 80507 56409 81501 36281  \
SUBJECT_ID                                                                      
4074          0    0     0    0     0     0    0    0     0     0     0     0   
90889         0    0     0    0     0     0    0    0     0     0     0     0   
72753         0    0     0    0     0     0    0    0     0     0     0     0   
64908         0    0     0    0     0     0    0    0     0     0     0     0   
70273         0    0     0    0     0     0    0    0     0     0     0     0   

           77083 86803 78951 E9330 E8782 4210 80842 7856 8670 78659 7249  \
SUBJECT_ID                                                                 
4074           0     0     0     0     0    0     0    0    0     0    0   
90889          0     0     0     0     1    0     0    0    0     0    0   
72753          0     0     0     0     0    0     0    0    0     0    0   
64908          0     0     0     0     0    0     0    0    0     0    0   
70273          0     0     0     0     0    0     0    0    0     0    0   

           27669  5110 0971 5070 6000 86403 70709 5302 V1041 5722 6805 30391  \
SUBJECT_ID                                                                     
4074           0     0    0    0    0     0     0    0     0    0    0     0   
90889          0     0    0    0    0     0     0    0     0    0    0     0   
72753          0     0    0    0    0     0     0    0     0    0    0     0   
64908          0     0    0    0    0     0     0    0     0    0    0     0   
70273          0     0    0    0    0     0     0    0     0    0    0     0   

           78821 76519 9534 7231 9708 5718 30591 36221 E8796 220 5759 V5867  \
SUBJECT_ID                                                                    
4074           0     0    0    0    0    0     0     0     0   0    0     0   
90889          0     0    0    0    0    0     0     0     0   0    0     0   
72753          0     0    0    0    0    0     0     0     0   0    0     0   
64908          0     0    0    0    0    0     0     0     0   0    0     0   
70273          0     0    0    0    0    0     0     0     0   0    0     0   

           56983 7242 2440 2409 5730 23871 03810 0090 70703 0311 5711 5401  \
SUBJECT_ID                                                                   
4074           0    0    0    0    0     0     0    0     0    0    0    0   
90889          0    0    0    0    0     0     0    0     0    0    0    0   
72753          0    0    0    0    0     0     0    0     0    0    0    0   
64908          0    0    0    0    0     0     0    0     0    0    0    0   
70273          0    0    0    0    0     0     0    0     0    0    0    0   

           53560 V3001 41001 45620 86504 78065 75470 1830 7245 8083 3051  \
SUBJECT_ID                                                                 
4074           0     0     0     0     0     0     0    0    0    0    0   
90889          0     0     0     0     0     0     0    0    0    0    0   
72753          0     0     0     0     0     0     0    0    0    0    0   
64908          0     0     0     0     0     0     0    0    0    0    0   
70273          0     0     0     0     0     0     0    0    0    0    0   

           78451 E9342 1561 30540 82521 43330 79319 1976 V430 496 41092 7824  \
SUBJECT_ID                                                                     
4074           0     0    0     0     0     0     0    0    0   0     0    0   
90889          0     0    0     0     0     0     0    0    0   0     0    0   
72753          0     0    0     0     0     0     0    0    0   0     0    0   
64908          0     0    0     0     0     0     0    0    0   1     0    0   
70273          0     0    0     0     0     0     0    0    0   1     0    0   

           5370 2639 99933 2532 41011 45340 1982 7756 67202 42789 78551 78079  \
SUBJECT_ID                                                               

## Convert patient-icd9 dataframe to a sparse matrix

In [18]:
# Convert to sparse DataFrame
sparse_df = icd9_df.astype(pd.SparseDtype("int", 0))
coo = sparse_df.sparse.to_coo()
csr = coo.tocsr()

# Print the shape of the sparse matrix
print(f"Sparse matrix shape: {csr.shape}")

x = 5
y = 15

# print csr matrix and compare with dataframe above to validate
print(f"First {x} rows and {y} columns of the sparse matrix:")
print(csr[:x, :y].toarray())

Sparse matrix shape: (930, 2259)
First 5 rows and 15 columns of the sparse matrix:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


## Calculate similarity between patients

In [19]:
# find the highest similarity between patients
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# calculate cosine similarity between patients
similarity_matrix = cosine_similarity(csr)

# convert to 3 decimal places
similarity_matrix = np.round(similarity_matrix, 3)

print(f"Shape of the similarity matrix: {similarity_matrix.shape}")
print(f"Similarity matrix:\n{similarity_matrix[:5, :5]}")

Shape of the similarity matrix: (930, 930)
Similarity matrix:
[[1.    0.116 0.202 0.148 0.148]
 [0.116 1.    0.101 0.    0.   ]
 [0.202 0.101 1.    0.065 0.065]
 [0.148 0.    0.065 1.    0.143]
 [0.148 0.    0.065 0.143 1.   ]]


## Select the top 3 matches for every patient

In [20]:
# Get the indices of the top 5 most similar patients for each patient
top_n = 3
top_n_indices = np.argsort(similarity_matrix, axis=1)[:, -top_n-1:-1][:, ::-1]

# Get the similarity scores of the top 3 most similar patients for each patient
# sort each row in ascending order, select highest 3 scores (except the last one which is always 1), and reverse the order to descending
top_n_scores = np.sort(similarity_matrix, axis=1)[:, -top_n-1:-1][:, ::-1]

# Get the patient IDs for the top 3 most similar patients for each patient
top_n_patient_ids = np.array([patients_df['SUBJECT_ID'].iloc[i] for i in top_n_indices])

# Create a DataFrame to store the results
top_n_df = pd.DataFrame(top_n_patient_ids, columns=[f"TOP_{i+1}_PATIENT_ID" for i in range(top_n)])

# Add the similarity scores to the DataFrame
top_n_df['SIMILARITY_SCORE'] = top_n_scores[:, 0]

# Add the patient IDs of the original patients to the DataFrame
top_n_df['PATIENT_ID'] = patients_df['SUBJECT_ID'].values

# Add the index of the original patients to the DataFrame
top_n_df['PATIENT_IDX'] = np.arange(len(patients_df))

# reorder the columns in place
top_n_df = top_n_df[['PATIENT_IDX', 'PATIENT_ID', 'SIMILARITY_SCORE'] + [f"TOP_{i+1}_PATIENT_ID" for i in range(top_n)]]

# Print the DataFrame
top_n_df.head(5)

,PATIENT_IDX,PATIENT_ID,SIMILARITY_SCORE,TOP_1_PATIENT_ID,TOP_2_PATIENT_ID,TOP_3_PATIENT_ID
0,0,4074,0.471,10139,17112,53836
1,1,90889,0.463,27725,64123,6338
2,2,72753,0.556,81074,96435,9652
3,3,64908,0.303,23867,10150,29902
4,4,70273,0.303,23867,75670,17399


## Create Similarity Network Graph

In [21]:
# convert top_n_df to a networkx graph
import networkx as nx

# Create a new graph
G_sim = nx.Graph()

# Add nodes for each patient
for i, row in top_n_df.iterrows():
    G_sim.add_node(int(row['PATIENT_ID']), similarity=row['SIMILARITY_SCORE'])

# Add edges between patients based on diagnosis
for i, row in top_n_df.iterrows():
    for j in range(1, top_n + 1):
        patient_id = int(row[f"TOP_{j}_PATIENT_ID"])
        G_sim.add_edge(int(row['PATIENT_ID']), patient_id, similarity=row['SIMILARITY_SCORE'])

# Print the number of nodes and edges in the graph
print(f"Number of nodes in the similarity graph: {G_sim.number_of_nodes()}")
print(f"Number of edges in the similarity graph: {G_sim.number_of_edges()}")


Number of nodes in the similarity graph: 930
Number of edges in the similarity graph: 2322


### View matplotlib of networkx similarity graph

In [22]:
# Draw the graph using matplotlib
# import matplotlib.pyplot as plt

# plt.figure(figsize=(6, 6))
# nx.draw(G_sim, with_labels=True, node_size=500, node_color="lightblue", font_size=10)
# plt.title("Similarity Graph")
# plt.show()

# Save Graph Network Files

## File Constants

In [23]:
import os

CURR_DIR_PATH = os.getcwd()

PICKLE_FILE_NAME = None
PICKLE_SIMILARITY_FILE_NAME = None
HTML_FILE_NAME = None
HTML_SIMILARITY_FILE_NAME = None

if G.number_of_nodes() < 10:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10_nodes.html"
    PICKLE_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10_nodes_similar.gpickle"
    HTML_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10_nodes_similarity.html"
elif G.number_of_nodes() < 50:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_50_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_50_nodes.html"
    PICKLE_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_50_nodes_similar.gpickle"
    HTML_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_50_nodes_similarity.html"
elif G.number_of_nodes() < 100:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_100_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_100_nodes.html"
    PICKLE_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_100_nodes_similar.gpickle"
    HTML_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_100_nodes_similarity.html"
elif G.number_of_nodes() < 1000:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_1000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_1000_nodes.html"
    PICKLE_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_1000_nodes_similar.gpickle"
    HTML_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_1000_nodes_similarity.html"
elif G.number_of_nodes() < 10000:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10000_nodes.html"
    PICKLE_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10000_nodes_similar.gpickle"
    HTML_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10000_nodes_similarity.html"
else:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_46000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_46000_nodes.html"
    PICKLE_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_46000_nodes_similar.gpickle"
    HTML_SIMILARITY_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_46000_nodes_similarity.html"

print(f"Pickle file name: {PICKLE_FILE_NAME}")
print(f"HTML file name: {HTML_FILE_NAME}")

Pickle file name: d:\Repos\ut-health-final-proj\pickle\patients_graph_max_1000_nodes.gpickle
HTML file name: d:\Repos\ut-health-final-proj\pyvis\graph_max_1000_nodes.html


## Pyvis Graph

In [24]:
# use pyvis to draw the graph
from pyvis.network import Network

print(f"Number of nodes in G: {len(G.nodes())}")
print(f"Number of edges in G: {len(G.edges())}")


if CREATE_PYVIS_GRAPHS:

    nt = Network(height="1200px", width="100%", notebook=False, directed=False, cdn_resources="remote")

    itr = G.nodes(data=True)
    count = 0
    print("\tNodes in the graph:")
    for node in itr:

        # some prints to help understand the structure of the nodes
        if count < 3:
            print(f"\t\t{node}")
        count += 1
        nt.add_node(node[0], label=node[0], title=f'Gender:{node[1]["gender"]}\nAge Bucket:{node[1]["age_bucket"]}\nHADM ID:{node[1]["hadm_id"]}', color='blue', size=10)

    itr = G.edges(data=True)
    count = 0
    print("\tEdges in the graph:")
    for edge in itr:

        # some prints to help understand the structure of the edges
        if count < 3:
            print(f"\t\t{edge}")
        count += 1

        if edge[2]['type'] == 'diagnosis':
            nt.add_edge(edge[0], edge[1], label=edge[2]['short_title'], title=edge[2]['icd9'], color='blue')
        elif edge[2]['type'] == 'procedures':
            nt.add_edge(edge[0], edge[1], label=edge[2]['short_title'], title=edge[2]['icd9'], color='orange')
        else:
            print("---- Edge not added, type not recognized ----")

    print("Number of nodes from networkx:", len(nt.nodes))
    print("Number of edges from networkx:", len(nt.edges))

    print("added nx graph to pyvis network")
    print(f"file name: {HTML_FILE_NAME}")
    # nt.toggle_physics(False)
    nt.show_buttons()
    nt.generate_html(HTML_FILE_NAME)
    nt.save_graph(HTML_FILE_NAME)
    # nt.show(HTML_FILE_NAME)
    print("saved pyvis graph to file")
else:
    print("Skipping pyvis graph creation")


Number of nodes in G: 930
Number of edges in G: 340365
Skipping pyvis graph creation


## Pickle File for Graph

In [25]:
import pickle

# Save the graph to a pickle file
with open(f"{PICKLE_FILE_NAME}", "wb") as f:
    pickle.dump(G, f)
    print("Graph saved to pickle file")

Graph saved to pickle file


# Save Graph Similarity Files

## Pyvis Similarity Graph

In [26]:
# use pyvis to draw the graph
from pyvis.network import Network

if CREATE_PYVIS_GRAPHS:

    # convert G_sim to a pyvis graph
    nt_sim = Network(height="1200px", width="100%", notebook=False, directed=False, cdn_resources="remote")
    nt_sim.from_nx(G_sim)

    # add node attributes to the pyvis network
    for node in nt_sim.nodes:
        node['title'] = f"Patient ID: {node['id']}\nSimilarity: {node['similarity']}"
        node['label'] = f"{node['id']}"
        node['font'] = {'size': 5}
    # add edge attributes to the pyvis network

    for edge in nt_sim.edges:
        edge['title'] = f"Similarity: {edge['similarity']}"
        edge['label'] = f"{edge['similarity']}"
        # set font size to 5
        edge['font'] = {'size': 5}

    print("added nx graph to pyvis network")
    print(f"file name: {HTML_SIMILARITY_FILE_NAME}")
    nt_sim.toggle_physics(False)
    nt_sim.show_buttons()
    nt_sim.generate_html(HTML_SIMILARITY_FILE_NAME)
    nt_sim.save_graph(HTML_SIMILARITY_FILE_NAME)
    print("saved pyvis graph to file")

else:
    print("Skipping pyvis similarity graph creation")

Skipping pyvis similarity graph creation


## Pickle Similarity Graph

In [27]:
import pickle

# Save the graph to a pickle file
with open(f"{PICKLE_SIMILARITY_FILE_NAME}", "wb") as f:
    pickle.dump(G_sim, f)
    print("Graph saved to pickle file")


Graph saved to pickle file


# Load Graph Netwprk from Pickle

In [28]:
# Load the graph from a pickle file
with open(f"{PICKLE_FILE_NAME}", "rb") as f:
    G = pickle.load(f)
    print("Graph loaded from pickle file")

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Graph loaded from pickle file
Number of nodes: 930
Number of edges: 340365


# Debug

## View Nodes: Connected Network X Graph

In [29]:
iter = G.nodes(data=True)
count = 0

print("Each node has the patient subject id, followed with a dictionary")
print("Nodes in the networkx graph:")
for node in iter:
    # some prints to help understand the structure of the nodes
    print(f"\t{node}")
    count += 1
    if count > 5:
        break

Each node has the patient subject id, followed with a dictionary
Nodes in the networkx graph:
	(4074, {'gender': 'M', 'age_bucket': 80, 'hadm_id': 137421})
	(90889, {'gender': 'M', 'age_bucket': 55, 'hadm_id': 168745})
	(72753, {'gender': 'M', 'age_bucket': 85, 'hadm_id': 144029})
	(64908, {'gender': 'M', 'age_bucket': 75, 'hadm_id': 172851})
	(70273, {'gender': 'F', 'age_bucket': 75, 'hadm_id': 102912})
	(92025, {'gender': 'F', 'age_bucket': 95, 'hadm_id': 160456})


## View Edges: Connected Network X Graph

In [30]:
iter = G.edges(data=True)
count = 0

print("Each edge has the source patient subject id, and the target patient subject id followed with a dictionary")
print("Edges in the networkx graph:")
for edge in iter:
    # some prints to help understand the structure of the edges
    print(f"\t{edge}")
    count += 1
    if count > 5:
        break

Each edge has the source patient subject id, and the target patient subject id followed with a dictionary
Edges in the networkx graph:
	(4074, 29184, {'icd9': 9907, 'short_title': 'Serum transfusion NEC', 'color': 'orange', 'type': 'procedures'})
	(4074, 29184, {'icd9': '42731', 'short_title': 'Atrial fibrillation', 'color': 'blue', 'type': 'diagnosis'})
	(4074, 90889, {'icd9': 9907, 'short_title': 'Serum transfusion NEC', 'color': 'orange', 'type': 'procedures'})
	(4074, 90889, {'icd9': '2724', 'short_title': 'Hyperlipidemia NEC/NOS', 'color': 'blue', 'type': 'diagnosis'})
	(4074, 25612, {'icd9': 9907, 'short_title': 'Serum transfusion NEC', 'color': 'orange', 'type': 'procedures'})
	(4074, 5774, {'icd9': 9907, 'short_title': 'Serum transfusion NEC', 'color': 'orange', 'type': 'procedures'})


## View Nodes: Similarity Network X Graph

In [31]:
iter = G_sim.nodes(data=True)
count = 0

print("Nodes in the networkx similarity graph:")
for node in iter:
    # some prints to help understand the structure of the nodes
    print(f"\t{node}")
    count += 1
    if count > 5:
        break

Nodes in the networkx similarity graph:
	(4074, {'similarity': np.float64(0.471)})
	(90889, {'similarity': np.float64(0.463)})
	(72753, {'similarity': np.float64(0.556)})
	(64908, {'similarity': np.float64(0.303)})
	(70273, {'similarity': np.float64(0.303)})
	(92025, {'similarity': np.float64(0.289)})


## View Edges: Similarity networkx graph

In [32]:
iter = G_sim.edges(data=True)
count = 0

print("Each edge has the source patient subject id, and the target patient subject id followed with the similarity score between the two patients")
print("Edges in the networkx graph:")
for edge in iter:
    # some prints to help understand the structure of the edges
    print(f"\t{edge}")
    count += 1
    if count > 5:
        break

Each edge has the source patient subject id, and the target patient subject id followed with the similarity score between the two patients
Edges in the networkx graph:
	(4074, 10139, {'similarity': np.float64(0.485)})
	(4074, 17112, {'similarity': np.float64(0.445)})
	(4074, 53836, {'similarity': np.float64(0.471)})
	(4074, 47937, {'similarity': np.float64(0.286)})
	(4074, 22057, {'similarity': np.float64(0.485)})
	(4074, 87955, {'similarity': np.float64(0.337)})
